In [ ]:
import numpy as np
from imputers.CSDI import CSDIImputer



CSDI_model = CSDIImputer(1)

CSDI_model.train(
    #training_data,
    masking ='rm',
    missing_ratio_or_k = 0.2,
    trainset_path = './datasets/train_ptbxl_248.npy',
    testset_path = './datasets/test_ptbxl_248.npy',
    epochs = 200,
    samples_generate = 10,
    path_save = "./results/train_ptbxl_248/CSDI",
    batch_size = 8,
    lr = 1.0e-3,
    layers = 4,
    channels = 64,
    nheads = 8,
    difussion_embedding_dim = 128,
    beta_start = 0.0001,
    beta_end = 0.5,
    num_steps = 50,
    schedule = 'quad',
    is_unconditional = 0,
    timeemb = 128,
    featureemb = 16,
    target_strategy = 'random')

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


{
    "train": {
        "epochs": 200,
        "batch_size": 8,
        "lr": 0.001,
        "path_save": "./results/train_ptbxl_248/CSDI"
    },
    "diffusion": {
        "layers": 4,
        "channels": 64,
        "nheads": 8,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 50,
        "schedule": "quad"
    },
    "model": {
        "missing_ratio_or_k": 0.2,
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 16,
        "target_strategy": "random",
        "masking": "rm"
    }
}
configuration file name: ./results/train_ptbxl_248/CSDIconfig_csdi_training


/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


4249.2026
2720.7368
3630.0532
3106.098
2518.9683
1313.5471
982.5496
1661.8728
1195.0836
798.6864
915.68256
662.1572
966.1312
1106.9817
578.4689
960.746
735.0709
680.2981
679.3266
765.66254
459.14584
450.39655
691.4521
345.9931
428.94547
876.0166
402.1497
414.09818
401.34778
365.36856
266.7741
371.87262
308.94733
222.4805
425.4207
505.75128
487.59506
567.7488
553.2813
704.8114
581.49243
588.3621
441.75665
467.46732
438.68924
423.8996
163.93585
226.45944
391.27347
352.71472
378.78406
528.4334
265.51608
688.38165
214.75095
261.42557
174.4311
536.5925
390.16376
607.9629
718.467
404.88654
506.8256
273.75763
259.56516
175.31192
739.6799
136.45386
129.94157
308.85907
278.36554
265.24127
386.34183
724.19385
597.0027
586.23596
74.04727
600.1893
360.21542
328.2155
396.12015
217.7215
236.5259
433.46954
293.7937
168.91989
322.7906
385.65637
560.3941
521.3928
145.87682
410.92114
114.76137
382.3011
168.36136
349.43597
463.32974
509.83447
496.34503
340.39783
221.1121
131.55063
209.11433
353.51715
387

In [3]:
import tensorflow as tf
#import tensorflow.eager as tfe
import os
#tf.enable_eager_execution()   # 开起eager模式，博主用的1.13.1，如果使用的是2.0，则忽略这行
#from read import _parse_function as _parse_function

#file_path = r'E:\dataset\DAVIS\JPEGImages\480p\bear'
#data = [os.path.join(file_path,i) for i in os.listdir(file_path)]
#label = [0]*len(data)
import numpy as np
#from imputers.CSDI import CSDI



path = './datasets/train_ptbxl_248.npy'

training_data = np.load(path) # 8000,100,14
training_data = training_data[4:,::]
#training_data = np.split(training_data, 2180, 0) # 160,50,100,14
#training_data = np.array(training_data)
training_data = training_data.reshape((2180*32,248,12))
#training_data = tf.convert_to_tensor(training_data)
#print('Data loaded')


In [11]:
def mask_missing_train_rm(data, missing_ratio=0.0):
    observed_values = np.array(data)
    observed_masks = ~np.isnan(observed_values)

    masks = observed_masks.reshape(-1).copy()
    obs_indices = np.where(masks)[0].tolist()
    miss_indices = np.random.choice(obs_indices, int(len(obs_indices) * missing_ratio), replace=False)
    masks[miss_indices] = False
    gt_masks = masks.reshape(observed_masks.shape)
    observed_values = np.nan_to_num(observed_values)
    observed_masks = observed_masks.astype("float32")
    gt_masks = gt_masks.astype("float32")

    return observed_values, observed_masks, gt_masks


In [12]:
def get_dataloader_train_impute(series,
                                batch_size=16,
                                missing_ratio_or_k=0.2,
                                len_dataset=248,
                                masking='rm',
                               path_save='',
                               ms=None):
   
    all_observed_values = []
    all_observed_masks = []
    all_gt_masks = []
    all_timepoints = []

    
    for sample in series:
        if masking == 'rm':
            #sample = sample.detach().cpu().numpy()
            observed_values, observed_masks, gt_masks = mask_missing_train_rm(sample, missing_ratio_or_k)
            #observed_values = torch.from_numpy(observed_values).cuda()
            #observed_masks = torch.from_numpy(observed_masks).cuda()
            #gt_masks = torch.from_numpy(gt_masks).cuda()
        elif masking == 'nrm':
            sample = sample.detach().cpu().numpy()
            observed_values, observed_masks, gt_masks = mask_missing_train_nrm(sample, missing_ratio_or_k)
            observed_values = torch.from_numpy(observed_values).cuda()
            observed_masks = torch.from_numpy(observed_masks).cuda()
            gt_masks = torch.from_numpy(gt_masks).cuda()
        elif masking == 'bm':
            sample = sample.detach().cpu().numpy()
            observed_values, observed_masks, gt_masks = mask_missing_train_bm(sample, missing_ratio_or_k)
            observed_values = torch.from_numpy(observed_values).cuda()
            observed_masks = torch.from_numpy(observed_masks).cuda()
            gt_masks = torch.from_numpy(gt_masks).cuda()

        all_observed_values.append(observed_values)
        all_observed_masks.append(observed_masks)
        all_gt_masks.append(gt_masks)
        all_timepoints.append(np.arange(len_dataset))
    
    dataset = tf.data.Dataset.from_tensor_slices((all_observed_values,all_observed_masks,all_gt_masks,all_timepoints))
    dataset = dataset.batch(16)
    
    return dataset


In [13]:
dataset_train = get_dataloader_train_impute(series=training_data)

In [17]:



iterator = iter(dataset_train)

while 1:
    try:
        no,image = enumerate(next(iterator))
        print(no)
        print(image[0].shape)
        print(image[1].shape)
        print(image[2].shape)
        print(image[3].shape)
    except StopIteration:   # python内置的迭代器越界错误类型
        print("iterator done")
        break;


ValueError: too many values to unpack (expected 2)

In [3]:
from tensorflow_models import nlp

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.12.0-dev20221203). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


AttributeError: partially initialized module 'cv2' has no attribute '_registerMatType' (most likely due to a circular import)

In [3]:
!pip install tf-models-official

     |████████████████████████████████| 2.3 MB 16.0 MB/s eta 0:00:01
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)
  Using cached Cython-0.29.32-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl (61 kB)
     |████████████████████████████████| 5.8 MB 66.6 MB/s eta 0:00:01
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Using cached kaggle-1.5.12.tar.gz (58 kB)
     |████████████████████████████████| 38.2 MB 64.3 MB/s eta 0:00:01
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached immutabledict-2.2.3-py3-none-any.whl (4.0 kB)
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
  Using cached tensorflow_model_optimization-0.7.3-py2.py3-none-any.whl (238 kB)
  Using cached pycocotools-2.0.6.tar.gz (24 kB)
  Installing build dependencies ... 

In [2]:
from tensorflow_models import nlp

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.12.0-dev20221203). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


TypeError: 'ABCMeta' object is not subscriptable

In [18]:
!pip uninstall tf-models-official

Found existing installation: tf-models-official 2.11.0
Uninstalling tf-models-official-2.11.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/official/*
    /usr/local/lib/python3.8/dist-packages/orbit/*
    /usr/local/lib/python3.8/dist-packages/tensorflow_models/*
    /usr/local/lib/python3.8/dist-packages/tf_models_official-2.11.0.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.8/dist-packages/official/nlp/modeling/networks/sparse_mixer.py
    /usr/local/lib/python3.8/dist-packages/official/nlp/modeling/networks/sparse_mixer_test.py
    /usr/local/lib/python3.8/dist-packages/official/projects/yolo/modeling/factory_test.py
    /usr/local/lib/python3.8/dist-packages/official/vision/losses/maskrcnn_losses_test.py
    /usr/local/lib/python3.8/dist-packages/official/vision/losses/segmentation_losses_test.py
    /usr/local/lib/python3.8/dist-packages/official/vision/modeling/layers/edgetpu.py
    /usr/local/lib/python3.8/dist-packages/offic

In [19]:
!Y

/bin/dash: 1: Y: not found


In [1]:
import tensorflow_addons as tfa

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.12.0-dev20221203). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


In [11]:
!pip install -q opencv-python

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [16]:
!pip3 install tf-models-nightly

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import tensorflow as tf

from tensorflow_models import nlp

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.12.0-dev20221203). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


In [1]:
import tensorflow as tf
tf.math.reduce_sum(tf.convert_to_tensor([[2,3],[1,4]]))

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [4]:
test=tf.Variable([[2,3],[1,4]])

In [5]:
test[1,1].assign(0)

<tf.Variable 'UnreadVariable' shape=(2, 2) dtype=int32, numpy=
array([[2, 3],
       [1, 0]], dtype=int32)>

In [2]:
import tensorflow_models

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
tf.__version__

'2.11.0'

In [4]:
import tensorflow as tf
build = tf.sysconfig.get_build_info()
print(build['cuda_version'])
print(build['cudnn_version'])

11.2
8


In [6]:
cudnn.__version__

NameError: name 'cudnn' is not defined